### Machine_translation (English to French)
#### 1. 데이터 불러오기

In [11]:
import collections
from jupyter_helpers import *
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
# GPU check
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [13]:
import os

def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r", encoding='utf-8') as f:
        data = f.read()

    return data.split('\n')

In [15]:
# Load English data
english_sentences = load_data('C:/Users/mark/Desktop/machine_translator_project/data/small_vocab_en')
# Load French data
french_sentences = load_data('C:/Users/mark/Desktop/machine_translator_project/data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


#### 2. small_vocab_en, small_vocab_fr 데이터 첫번째, 두번째 줄 확인

In [16]:
for sample_i in range(2):
    print('small_vocab_en {}번째 줄:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_kr {}번째 줄:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en 1번째 줄:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_kr 1번째 줄:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en 2번째 줄:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_kr 2번째 줄:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


#### 학습하는 단어의 수가 많을 수록(다양할수록) 각 특성을 파악하는 일이 많아진다.
#### 3. 데이터 셋을 파악한다.

In [17]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{}개의 영단어'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{}개의 유일한 영단어'.format(len(english_words_counter)))
print('가장 출현빈도가 높은 10개의 영단어: ')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()

print('{}개의 프랑스어단어'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{}개의 유일한 프랑스어단어'.format(len(french_words_counter)))
print('가장 출현빈도가 높은 10개의 프랑스어 단어: ')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250개의 영단어
227개의 유일한 영단어
가장 출현빈도가 높은 10개의 영단어: 
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295개의 프랑스어단어
355개의 유일한 프랑스어단어
가장 출현빈도가 높은 10개의 프랑스어 단어: 
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


#### 4. 단어의 토큰화(Tokenize)
#### keras의 tokenizer 기능을 사용하여 각 문장을 단어ID 값으로 변환한다.

In [19]:
def tokenize(x):
  tknzer = Tokenizer()
  tknzer.fit_on_texts(x)
  data = tknzer.texts_to_sequences(x) 

  return data, tknzer

text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


#### 5. Padding 
#### 다양한 문장의 길이를 동일한 길이로 통일시킨다. 각 시퀀스의 끝에 패딩을 추가하였다.

In [20]:
def pad(x, length=None):
    
  if length == None: 
    length = max(map(len, x))
    padded = pad_sequences(x, maxlen=length, padding='post')
  else:
    padded = pad_sequences(x, maxlen=length, padding='post')

  return np.array(padded) 

test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1)) # 내가 원하는 결과 예시: [1 2 4 5 6 7 8 9 0]
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


#### 6. 전처리 파이프라인은 preprocess 함수를 활용하였다.

In [21]:
def preprocess(x, y):
    
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("최대 영어 문장 길이:", max_english_sequence_length)
print("최대 프랑스어 문장 길이:", max_french_sequence_length)
french_vocab_size += 1
english_vocab_size += 1
print("영어 어휘 사전 크기:", english_vocab_size) #200
print("프랑스어 어휘 사전 크기:", french_vocab_size) #345

Data Preprocessed
최대 영어 문장 길이: 15
최대 프랑스어 문장 길이: 21
영어 어휘 사전 크기: 200
프랑스어 어휘 사전 크기: 345


#### 7. 단어ID 값을 프랑스어로 출력

In [24]:
def logits_to_text(logits, tokenizer):
    
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


#### 8. Model
#### simple-RNN, Embedding, Bidirectional RNN, encoder-decoder RNN 4가지 구현

#### (1) simple-RNN

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dropout

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(0.005),
                  metrics=['accuracy'])
    
    
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 159s 1s/step - loss: 2.5825 - accuracy: 0.4695 - val_loss: 1.3003 - val_accuracy: 0.6323
Epoch 2/10
108/108 [==============================] - 152s 1s/step - loss: 1.2925 - accuracy: 0.6303 - val_loss: 1.0747 - val_accuracy: 0.6771
Epoch 3/10
108/108 [==============================] - ETA: 0s - loss: 1.1061 - accuracy: 0.6640

In [ ]:
# simple RNN 구조보기
print("Prediction:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

#### (2) Embedding

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(0.005),
                  metrics=['accuracy'])
    return model


tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

#### (3) Bidirectional RNNs

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

bd_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

bd_rnn_model.summary()

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

#### (4) Encoder-decoder

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    model = Sequential()
    # Encoder
    model.add(GRU(256, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(0.001),
                  metrics=['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

#### 9. Prediction

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

def final_predictions_for_encdec_model(x, y, x_tk, y_tk):
    
    callback = [EarlyStopping(monitor='val_accuracy', patience=3, mode='max')]
    
    max_y_sequence_length = len(y[0])
    x_vocab_size = len(x_tk.word_index)
    y_vocab_size = len(y_tk.word_index)
    
    model = model_final(x.shape, y.shape[1],
                            x_vocab_size+1, y_vocab_size+1)

    model.sumarry()
    model.fit(x, y, batch_size=1024, epochs=25, validation_split=0.2)

    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    
    def sentence_to_model_input(sentences):
        sentences_decoder = pad(x, max_y_sequence_length)
        
        return [sentences, sentences_decoder]
    
    predictions = model.predict(sentence_to_model_input(sentences), len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions_for_encdec_model(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)